# **Taxi Driver Predictions**

## **Import Packages and Data**

In [1]:
# Import packages
import pandas as pd 
import matplotlib.pyplot as plt

In [7]:
# Read data
taxi = pd.read_csv('/Users/kellyshreeve/Desktop/Data-Sets/taxi.csv', parse_dates=[0], index_col=[0])

In [8]:
# Print info
print(taxi.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB
None


Index is datetime and only one column with number of taxi orders. There are no missing values.

In [9]:
# Display head
display(taxi.head(10))

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32
2018-03-01 00:50:00,21
2018-03-01 01:00:00,7
2018-03-01 01:10:00,5
2018-03-01 01:20:00,17


In [13]:
print(taxi.index.is_monotonic)

AttributeError: 'DatetimeIndex' object has no attribute 'is_monotonic'

### Initial data observations

The dataset contains taxi orders in 10-minute intervals. There is only one column with number of taxi orders and 26496 observations. There are no missing values. The data is ready for analysis.

## **Exploratory Analysis**

In [ ]:
# Plot time series
